In [26]:
import pandas as pd

mouse_data = pd.read_csv("mouse_combined.csv")
keyboard_data = pd.read_csv("keyboard_combined.csv")

In [55]:
from sklearn.model_selection import train_test_split

X_keyboard = keyboard_data.drop(columns=["is_bot"])
y_keyboard = keyboard_data["is_bot"]
X_train_keyboard, X_test_keyboard, y_train_keyboard, y_test_keyboard = train_test_split(
    X_keyboard, y_keyboard, test_size=0.2, random_state=42
)

X_mouse = mouse_data.drop(columns=["is_bot","session_id", "event_type"])
y_mouse = mouse_data["is_bot"]
X_train_mouse, X_test_mouse, y_train_mouse, y_test_mouse = train_test_split(
    X_mouse, y_mouse, test_size=0.2, random_state=42
)


In [29]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate

keyboard_input = Input(shape=(X_train_keyboard.shape[1],), name="keyboard_input")
keyboard_dense = Dense(64, activation="relu")(keyboard_input)

mouse_input = Input(shape=(X_train_mouse.shape[1],), name="mouse_input")
mouse_dense = Dense(128, activation="relu")(mouse_input)

merged = Concatenate(name="merged_layer")([keyboard_dense, mouse_dense])
output = Dense(1, activation="sigmoid", name="output")(merged)

model = Model(inputs=[keyboard_input, mouse_input], outputs=output)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ keyboard_input (InputLayer)   │ (None, 26)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ mouse_input (InputLayer)      │ (None, 3)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_8 (Dense)               │ (None, 64)                │           1,728 │ keyboard_input[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_9 (Dense)               │ (None, 128)               │             512 │ mouse_input[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ merged_layer (Concatenate)    │ (None, 192)               │               0 │ dense_8[0][0],             │
│                               │                           │                 │ dense_9[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ output (Dense)                │ (None, 1)                 │             193 │ merged_layer[0][0]         │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,433 (9.50 KB)

 Trainable params: 2,433 (9.50 KB)

 Non-trainable params: 0 (0.00 B)

In [54]:
from sklearn.utils import resample

X_train_mouse_resampled = X_train_mouse[:len(X_train_keyboard)]
y_train_mouse_resampled = y_train_mouse[:len(X_train_keyboard)]
